# Fibonacci - Recursive implementation

In [ ]:
from typing import Callable, Dict
from functools import lru_cache

The first task is to find the nth finonacci number. Start with some test data that represents where we want to get to

In [ ]:
FIXTURES = [
    (1, 1,),
    (2, 1,),
    (3, 2,),
    (5, 5,),
    (6, 8,),
    (7, 13,),
    (8, 21,),
    (12, 144,),
    (15, 610,),
]

In [ ]:
def test_fib(fib: Callable):
    for input_value, expected_value in FIXTURES:
        output = fib(input_value)
        print(f'fib({input_value}) = {output}: expected {expected_value}')

Our first implementation uses recursion

In [ ]:
def fib(n: int) -> int:
    '''
    Computes the nth fibonacci number where F(n) = F(n-1) + F(n-2)
    '''
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)

In [ ]:
test_fib(fib)

This works, but doesn't scale

In [ ]:
FIXTURES.append((50, 12586269025,))
# this will take too long
test_fib(fib)

The problem is the depth of recursive calls and the fact that sub-problems are computed over and over again:

![Call structure of our initial fibonacci implementation](img/fib_6.png)

To avoid this issue we use a technique called `memoization` where a value is only computed once. In subsequent invocations it is retrieved from a cache

In [ ]:
def fib(n: int, memo: Dict={}) -> int:
    '''
    Computes the nth fibonacci number
    '''
    # is it cached?
    if n in memo:
        return memo[n]
    if n <= 2:
        return 1
    # calculate and store the nth number
    # uncomment this line to see the effect of having memo as a mutable parameter
    # print(f'Calculating F({n})')
    memo[n] = fib(n - 1, memo) + fib(n - 2, memo)
    return memo[n]

In [ ]:
test_fib(fib)

This is much faster but there are currently two problems with the implementation as it stands

1. The `memo` dict is part of the functions internal implementation, and should never be visible to clients
2. The provide a new dictionary instance as a default. However a `dict` is a *mutable* type than can be (and is) changed during function execution. That means that next time you call the function the dictionary will be in the state that it was when the previous call finished.

To see this type `fib.` followed by `shift-tab` to see the function signature and docstring. You will see that the `memo` argument now contains cached values from the earlier invocation.

You can also uncomment the line that prints when a fibonacci number is being computed (rather than retieved from the cache) and call the function twice.

In [ ]:
fib(10)

In [ ]:
fib(10)

Notice that the second time all values were retrieved from the cache. This is a side effect that can cause some very difficult and unexpected bugs. We shall see an example when we try and use the same technique in our grid traveller implementation

The solution is to remove the `memo` dict as a parameter and instead have it as an attribute visible only in the function itself. To achieve this we use and inner function:

In [ ]:
def fib(n: int) -> int:
    '''
    Computes the nth fibonacci number
    '''
    
    # this we be visible between all invocations of the inner function
    memo = {}

    def _fib(n: int) -> int:
        if n in memo:
            return memo[n]
        if n <= 2:
            return 1
        memo[n] = _fib(n - 1) + _fib(n - 2)
        return memo[n]
    # return the computed value to the client
    return _fib(n)

In [ ]:
test_fib(fib)

Extending a function with its own `dict` for caching can also be achived via a decorator:

In [ ]:
def memoize(f: Callable) -> Callable:
    '''
    Provides memoization for a decoration function
    '''
    _cache = {}
    
    # this is the inner function that
    # will be the one called after decoration
    # it calls the decorated function only when the value is not in the cache
    def wrapper(n: int) -> int:
        if n in _cache:
            return _cache[n]
        _cache[n] = f(n)
        return _cache[n]
    return wrapper

In [ ]:
# our original naive implementation memoized

@memoize
def fib(n: int) -> int:
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)

In [ ]:
test_fib(fib)

Our decorator provides memoization service for any function that takes a single integer argument. The `lru_cache` decorator in the standard library extends this to functions with any signature

In [ ]:
@lru_cache(maxsize=4096)
def fib(n: int) -> int:
    if n <= 2:
        return 1
    return fib(n - 1) + fib(n - 2)

In [ ]:
test_fib(fib)